In [1]:
import numpy as np
import pandas as pd
import nibabel as nib
import random
import sys
import os
import scipy
import os.path as osp
import torch

sys.path.append('..')

from functions.visualization import polarAngle_plot, signMap_plot
from deepRetinotopy_TheToolbox.utils.rois import ROI_WangParcelsPlusFovea as roi
from ipywidgets import interact, Dropdown

In [4]:
def get_giftis(hemisphere, prediction, path_to_freesurfer_dir, path_to_converted_data, mask = False):
    label_primary_visual_areas = ['ROI']
    
    mask_L, mask_R, index_L_mask, index_R_mask = roi(
        label_primary_visual_areas)
    number_cortical_nodes = int(64984)
    number_hemi_nodes = int(number_cortical_nodes / 2)
        
    for sub_id in os.listdir(path_to_freesurfer_dir):
        if hemisphere == 'rh':
            template = nib.load(osp.join(path_to_freesurfer_dir, sub_id + '/surf/' + sub_id + 
                                     '.curvature-midthickness.rh.32k_fs_LR.func.gii'))
            if prediction == 'polarAngle':
                polarAngle = scipy.io.loadmat(osp.join(path_to_converted_data, 
                                                        'cifti_polarAngle_all.mat'))['cifti_polarAngle']
                retinotopicMap_values = torch.tensor(np.reshape(
                    polarAngle['x' + str(sub_id) + 
                                '_fit1_polarangle_msmall'][0][0][number_hemi_nodes:number_cortical_nodes
                                                                ].reshape((number_hemi_nodes)), (-1, 1)
                                                                ),dtype=torch.float)
            elif prediction == 'eccentricity':
                eccentricity = scipy.io.loadmat(osp.join(path_to_converted_data, 
                                                         'cifti_eccentricity_all.mat'))['cifti_eccentricity']
                retinotopicMap_values = torch.tensor(np.reshape(
                    eccentricity['x' + str(sub_id) + 
                                 '_fit1_eccentricity_msmall'][0][0][number_hemi_nodes:number_cortical_nodes
                                                                    ].reshape((number_hemi_nodes)), (-1, 1)
                                                                    ),dtype=torch.float)
            elif prediction == 'pRFsize':
                prfsize = scipy.io.loadmat(osp.join(path_to_converted_data, 
                                                         'cifti_pRFsize_all.mat'))['cifti_pRFsize']
                retinotopicMap_values = torch.tensor(np.reshape(
                    prfsize['x' + str(sub_id) + 
                             '_fit1_receptivefieldsize_msmall'][0][0][number_hemi_nodes:number_cortical_nodes
                                                                    ].reshape((number_hemi_nodes)), (-1, 1)
                                                                    ),dtype=torch.float)
            if mask==True:
                retinotopicMap_values[mask_R==0] = -10
        elif hemisphere == 'lh':
            template = nib.load(osp.join(path_to_freesurfer_dir, sub_id + '/surf/' + sub_id + 
                                     '.curvature-midthickness.lh.32k_fs_LR.func.gii'))
            if prediction == 'polarAngle':
                polarAngle = scipy.io.loadmat(osp.join(path_to_converted_data, 
                                                        'cifti_polarAngle_all.mat'))['cifti_polarAngle']
                retinotopicMap_values = torch.tensor(np.reshape(
                    polarAngle['x' + str(sub_id) + '_fit1_polarangle_msmall'][0][
                        0][0:number_hemi_nodes].reshape((number_hemi_nodes)), (-1, 1)), dtype=torch.float)
                # Transform the values to match the prediction
                sum = retinotopicMap_values < 180
                minus = retinotopicMap_values > 180
                retinotopicMap_values[sum] = retinotopicMap_values[sum] + 180
                retinotopicMap_values[minus] = retinotopicMap_values[minus] - 180
            
            elif prediction == 'eccentricity':
                eccentricity = scipy.io.loadmat(osp.join(path_to_converted_data, 
                                                         'cifti_eccentricity_all.mat'))['cifti_eccentricity']
                retinotopicMap_values = torch.tensor(np.reshape(
                    eccentricity['x' + str(sub_id) + '_fit1_eccentricity_msmall'][
                        0][0][0:number_hemi_nodes].reshape((number_hemi_nodes)), (-1, 1)), dtype=torch.float)
            elif prediction == 'pRFsize':
                prfsize = scipy.io.loadmat(osp.join(path_to_converted_data, 
                                                         'cifti_pRFsize_all.mat'))['cifti_pRFsize']
                retinotopicMap_values = torch.tensor(np.reshape(
                    prfsize['x' + str(sub_id) + '_fit1_receptivefieldsize_msmall'][
                        0][0][0:number_hemi_nodes].reshape((number_hemi_nodes)), (-1, 1)), dtype=torch.float)
            if mask==True:
                retinotopicMap_values[mask_L==0] = -10

        retinotopicMap_values = np.reshape(retinotopicMap_values, (number_hemi_nodes,1))
        template.agg_data()[:] = np.reshape(retinotopicMap_values, (-1))

        if hemisphere == 'lh':
            if mask==False:
                nib.save(template, path_to_freesurfer_dir + sub_id + '/deepRetinotopy/' + sub_id + '.fs_empirical_' + prediction +
                                        '_lh.func.gii')
            if mask==True:
                nib.save(template, path_to_freesurfer_dir + sub_id + '/deepRetinotopy/' + sub_id + '.fs_empirical_' + prediction +
                                        '_lh_masked.func.gii')
        elif hemisphere == 'rh':
            if mask==False:
                nib.save(template, path_to_freesurfer_dir + sub_id + '/deepRetinotopy/' + sub_id + '.fs_empirical_' + prediction +
                                            '_rh.func.gii')  
            if mask==True:
                nib.save(template, path_to_freesurfer_dir + sub_id + '/deepRetinotopy/' + sub_id + '.fs_empirical_' + prediction +
                                        '_rh_masked.func.gii')
        print('Empirical map has been saved as ' + sub_id + '.fs_empirical_' + prediction + '_' + hemisphere + '.func.gii')
    return 


In [5]:
path_to_freesurfer_dir = '../../deepRetinotopy_TheToolbox/HCP/freesurfer/'
path_to_converted_data = '../../deepRetinotopy_TheToolbox/HCP/raw/converted/'
retinotopic_map = ['polarAngle', 'eccentricity', 'pRFsize']

for hemisphere in ['lh', 'rh']:
    for prediction in retinotopic_map:
        get_giftis(hemisphere, prediction, path_to_freesurfer_dir, path_to_converted_data, mask = False)
        get_giftis(hemisphere, prediction, path_to_freesurfer_dir, path_to_converted_data, mask = True)

Empirical map has been saved as 100610.fs_empirical_pRFsize_lh.func.gii
Empirical map has been saved as 140117.fs_empirical_pRFsize_lh.func.gii
Empirical map has been saved as 162935.fs_empirical_pRFsize_lh.func.gii
Empirical map has been saved as 197348.fs_empirical_pRFsize_lh.func.gii
Empirical map has been saved as 116726.fs_empirical_pRFsize_lh.func.gii
Empirical map has been saved as 159239.fs_empirical_pRFsize_lh.func.gii
Empirical map has been saved as 572045.fs_empirical_pRFsize_lh.func.gii
Empirical map has been saved as 233326.fs_empirical_pRFsize_lh.func.gii
Empirical map has been saved as 148133.fs_empirical_pRFsize_lh.func.gii
Empirical map has been saved as 187345.fs_empirical_pRFsize_lh.func.gii
Empirical map has been saved as 550439.fs_empirical_pRFsize_lh.func.gii
Empirical map has been saved as 826353.fs_empirical_pRFsize_lh.func.gii
Empirical map has been saved as 352738.fs_empirical_pRFsize_lh.func.gii
Empirical map has been saved as 617748.fs_empirical_pRFsize_lh.f

In [2]:
path_to_freesurfer = '../../deepRetinotopy_TheToolbox/HCP/freesurfer/'
# subject_id= os.listdir(path_to_freesurfer)
subject_id = ['186949', '169747', '826353', '825048', '671855',
                        '751550', '318637', '131722', '137128', '706040'] # dev set
# if index_sub = subject_id.index('processed') # to edit
# subject_id.pop(index_sub)
prediction = Dropdown(options = ['average', 'empirical','model1', 'model2', 'model3', 'model4', 'model5'])
binarize = Dropdown(options = [False, True])

@interact(subject_id=subject_id, prediction = prediction, binarize = binarize)
def plot1(subject_id,prediction, binarize):
    return polarAngle_plot(subject_id, path_to_freesurfer, '../../deepRetinotopy_TheToolbox/templates', prediction = prediction, binarize = binarize)

interactive(children=(Dropdown(description='subject_id', options=('186949', '169747', '826353', '825048', '671…